### Imports 

In [1]:
from sagemaker.tensorflow.serving import TensorFlowModel
from sagemaker.multidatamodel import MultiDataModel
from sagemaker.tensorflow import TensorFlow
from sagemaker.inputs import TrainingInput
from sagemaker import get_execution_role
from datetime import datetime
import tensorflow as tf
import sagemaker
import time
import boto3
import os

In [2]:
print(f'Using TensorFlow version: {tf.__version__}')
print(f'Using SageMaker version: {sagemaker.__version__}')

Using TensorFlow version: 2.3.0
Using SageMaker version: 2.16.1


### Essentials

In [3]:
role = get_execution_role()
session = boto3.Session()
sagemaker_session = sagemaker.Session()

s3 = session.resource('s3')
TF_FRAMEWORK_VERSION = '2.3.0'
BUCKET = 'tf-mme-892313895307' # USE YOUR ACCOUNT ID OR INITIALS AS SUFFIX
PREFIX = 'cv-models'

## Model 1 - CIFAR-10 Image Classification

### Copy Train & Validation Data to S3

In [4]:
!aws s3 cp ./DATA/CIFAR_10/train s3://{BUCKET}/{PREFIX}/cifar/train --recursive

upload: DATA/CIFAR_10/train/y_train.npy to s3://tf-mme-892313895307/cv-models/cifar/train/y_train.npy
upload: DATA/CIFAR_10/train/X_train.npy to s3://tf-mme-892313895307/cv-models/cifar/train/X_train.npy


In [5]:
!aws s3 cp ./DATA/CIFAR_10/validation s3://{BUCKET}/{PREFIX}/cifar/validation --recursive

upload: DATA/CIFAR_10/validation/y_validation.npy to s3://tf-mme-892313895307/cv-models/cifar/validation/y_validation.npy
upload: DATA/CIFAR_10/validation/X_validation.npy to s3://tf-mme-892313895307/cv-models/cifar/validation/X_validation.npy


In [6]:
!aws s3 cp ./DATA/CIFAR_10/test s3://{BUCKET}/{PREFIX}/cifar/test --recursive

upload: DATA/CIFAR_10/test/y_test.npy to s3://tf-mme-892313895307/cv-models/cifar/test/y_test.npy
upload: DATA/CIFAR_10/test/X_test.npy to s3://tf-mme-892313895307/cv-models/cifar/test/X_test.npy


In [7]:
train_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/cifar/train', 
                            distribution='FullyReplicated', 
                            content_type='npy')
validation_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/cifar/validation', 
                                 distribution='FullyReplicated', 
                                 content_type='npy')
test_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/cifar/test', 
                           distribution='FullyReplicated', 
                           content_type='npy')

In [8]:
inputs = {'train': train_input, 'val': validation_input, 'test': test_input}

### Estimator 1

In [9]:
model_name = 'model-1'
hyperparameters = {'epochs': 30}
estimator_parameters = {'entry_point':'cifar_train.py',
                        'instance_type': 'ml.m5.2xlarge',
                        'instance_count': 1,
                        'model_dir': f'/opt/ml/model',
                        'role': role,
                        'hyperparameters': hyperparameters,
                        'output_path': f's3://{BUCKET}/{PREFIX}/cifar/out',
                        'base_job_name': f'cv-{model_name}',
                        'framework_version': TF_FRAMEWORK_VERSION,
                        'py_version': 'py37',
                        'script_mode': True}
estimator_1 = TensorFlow(**estimator_parameters)

In [10]:
estimator_1.fit(inputs)

2020-11-27 20:14:12 Starting - Starting the training job...
2020-11-27 20:14:18 Starting - Launching requested ML instances......
2020-11-27 20:15:32 Starting - Preparing the instances for training...
2020-11-27 20:16:09 Downloading - Downloading input data...
2020-11-27 20:16:39 Training - Downloading the training image...
2020-11-27 20:16:57 Training - Training image download completed. Training in progress.2020-11-27 20:17:01,497 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2020-11-27 20:17:01,504 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-11-27 20:17:02,020 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-11-27 20:17:02,036 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-11-27 20:17:02,053 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-11-27 20:17:02,063 sagemaker-traini

1546/1546 - 41s - loss: 1.7426 - accuracy: 0.3775 - val_loss: 1.3958 - val_accuracy: 0.4780
Epoch 2/30
1546/1546 - 41s - loss: 1.5332 - accuracy: 0.4484 - val_loss: 1.3562 - val_accuracy: 0.5140
Epoch 3/30
1546/1546 - 41s - loss: 1.4758 - accuracy: 0.4654 - val_loss: 1.8080 - val_accuracy: 0.4320
Epoch 4/30
1546/1546 - 41s - loss: 1.4439 - accuracy: 0.4783 - val_loss: 1.2650 - val_accuracy: 0.5440
Epoch 5/30
1546/1546 - 41s - loss: 1.4260 - accuracy: 0.4906 - val_loss: 1.2867 - val_accuracy: 0.5500
Epoch 6/30
1546/1546 - 41s - loss: 1.3991 - accuracy: 0.4993 - val_loss: 1.3642 - val_accuracy: 0.5120
Epoch 7/30
1546/1546 - 40s - loss: 1.3806 - accuracy: 0.5096 - val_loss: 1.2061 - val_accuracy: 0.5780
Epoch 8/30
1546/1546 - 41s - loss: 1.3879 - accuracy: 0.5039 - val_loss: 1.2801 - val_accuracy: 0.5280
Epoch 9/30
1546/1546 - 41s - loss: 1.4048 - accuracy: 0.5009 - val_loss: 1.2346 - val_accuracy: 0.5360
Epoch 10/30
1546/1546 - 41s - loss: 1.3684 - accuracy: 0.5115 - val_loss: 1.2653 - v


2020-11-27 20:37:49 Uploading - Uploading generated training model
2020-11-27 20:37:49 Completed - Training job completed
Training seconds: 1300
Billable seconds: 1300


## Model 2 - Sign Language Classification 

### Copy Train & Validation Data to S3

In [11]:
!aws s3 cp ./DATA/SIGN_LANGUAGE/ s3://{BUCKET}/{PREFIX}/sign_lang --recursive

upload: DATA/SIGN_LANGUAGE/.DS_Store to s3://tf-mme-892313895307/cv-models/sign_lang/.DS_Store
upload: DATA/SIGN_LANGUAGE/test/0/IMG_4159.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/test/0/IMG_4159.JPG
upload: DATA/SIGN_LANGUAGE/test/.DS_Store to s3://tf-mme-892313895307/cv-models/sign_lang/test/.DS_Store
upload: DATA/SIGN_LANGUAGE/test/0/IMG_4976.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/test/0/IMG_4976.JPG
upload: DATA/SIGN_LANGUAGE/test/0/IMG_5695.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/test/0/IMG_5695.JPG
upload: DATA/SIGN_LANGUAGE/test/1/IMG_5992.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/test/1/IMG_5992.JPG
upload: DATA/SIGN_LANGUAGE/test/0/IMG_5572.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/test/0/IMG_5572.JPG
upload: DATA/SIGN_LANGUAGE/test/1/IMG_5401.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/test/1/IMG_5401.JPG
upload: DATA/SIGN_LANGUAGE/test/1/IMG_5340.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/test/1/IMG_5340

upload: DATA/SIGN_LANGUAGE/train/0/IMG_4149.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/0/IMG_4149.JPG
upload: DATA/SIGN_LANGUAGE/train/0/IMG_4129.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/0/IMG_4129.JPG
upload: DATA/SIGN_LANGUAGE/train/0/IMG_4203.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/0/IMG_4203.JPG
upload: DATA/SIGN_LANGUAGE/train/0/IMG_4109.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/0/IMG_4109.JPG
upload: DATA/SIGN_LANGUAGE/train/0/IMG_4213.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/0/IMG_4213.JPG
upload: DATA/SIGN_LANGUAGE/train/0/IMG_4119.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/0/IMG_4119.JPG
upload: DATA/SIGN_LANGUAGE/train/0/IMG_4223.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/0/IMG_4223.JPG
upload: DATA/SIGN_LANGUAGE/train/0/IMG_4170.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/0/IMG_4170.JPG
upload: DATA/SIGN_LANGUAGE/train/0/IMG_4233.JPG to s3://tf-mme-8

upload: DATA/SIGN_LANGUAGE/train/0/IMG_5132.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/0/IMG_5132.JPG
upload: DATA/SIGN_LANGUAGE/train/0/IMG_5122.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/0/IMG_5122.JPG
upload: DATA/SIGN_LANGUAGE/train/0/IMG_5101.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/0/IMG_5101.JPG
upload: DATA/SIGN_LANGUAGE/train/0/IMG_5111.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/0/IMG_5111.JPG
upload: DATA/SIGN_LANGUAGE/train/0/IMG_5173.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/0/IMG_5173.JPG
upload: DATA/SIGN_LANGUAGE/train/0/IMG_5152.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/0/IMG_5152.JPG
upload: DATA/SIGN_LANGUAGE/train/0/IMG_5142.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/0/IMG_5142.JPG
upload: DATA/SIGN_LANGUAGE/train/0/IMG_5183.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/0/IMG_5183.JPG
upload: DATA/SIGN_LANGUAGE/train/0/IMG_5193.JPG to s3://tf-mme-8

upload: DATA/SIGN_LANGUAGE/train/1/IMG_1220.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/1/IMG_1220.JPG
upload: DATA/SIGN_LANGUAGE/train/1/IMG_1200.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/1/IMG_1200.JPG
upload: DATA/SIGN_LANGUAGE/train/1/IMG_1129.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/1/IMG_1129.JPG
upload: DATA/SIGN_LANGUAGE/train/1/IMG_1180.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/1/IMG_1180.JPG
upload: DATA/SIGN_LANGUAGE/train/1/IMG_1230.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/1/IMG_1230.JPG
upload: DATA/SIGN_LANGUAGE/train/1/IMG_1119.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/1/IMG_1119.JPG
upload: DATA/SIGN_LANGUAGE/train/1/IMG_1210.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/1/IMG_1210.JPG
upload: DATA/SIGN_LANGUAGE/train/1/IMG_1190.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/1/IMG_1190.JPG
upload: DATA/SIGN_LANGUAGE/train/1/IMG_1271.JPG to s3://tf-mme-8

upload: DATA/SIGN_LANGUAGE/train/1/IMG_4827.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/1/IMG_4827.JPG
upload: DATA/SIGN_LANGUAGE/train/1/IMG_4837.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/1/IMG_4837.JPG
upload: DATA/SIGN_LANGUAGE/train/1/IMG_4897.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/1/IMG_4897.JPG
upload: DATA/SIGN_LANGUAGE/train/1/IMG_4857.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/1/IMG_4857.JPG
upload: DATA/SIGN_LANGUAGE/train/1/IMG_4847.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/1/IMG_4847.JPG
upload: DATA/SIGN_LANGUAGE/train/1/IMG_4887.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/1/IMG_4887.JPG
upload: DATA/SIGN_LANGUAGE/train/1/IMG_4927.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/1/IMG_4927.JPG
upload: DATA/SIGN_LANGUAGE/train/1/IMG_4967.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/1/IMG_4967.JPG
upload: DATA/SIGN_LANGUAGE/train/1/IMG_4987.JPG to s3://tf-mme-8

...

upload: DATA/SIGN_LANGUAGE/train/9/IMG_5254.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/9/IMG_5254.JPG
upload: DATA/SIGN_LANGUAGE/train/9/IMG_5307.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/9/IMG_5307.JPG
upload: DATA/SIGN_LANGUAGE/train/9/IMG_5265.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/9/IMG_5265.JPG
upload: DATA/SIGN_LANGUAGE/train/9/IMG_5327.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/9/IMG_5327.JPG
upload: DATA/SIGN_LANGUAGE/train/9/IMG_5338.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/9/IMG_5338.JPG
upload: DATA/SIGN_LANGUAGE/train/9/IMG_5222.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/9/IMG_5222.JPG
upload: DATA/SIGN_LANGUAGE/train/9/IMG_5349.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/9/IMG_5349.JPG
upload: DATA/SIGN_LANGUAGE/train/9/IMG_5379.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/train/9/IMG_5379.JPG
upload: DATA/SIGN_LANGUAGE/train/9/IMG_5389.JPG to s3://tf-mme-8

upload: DATA/SIGN_LANGUAGE/valid/0/IMG_5582.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/0/IMG_5582.JPG
upload: DATA/SIGN_LANGUAGE/valid/0/IMG_5497.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/0/IMG_5497.JPG
upload: DATA/SIGN_LANGUAGE/valid/0/IMG_5318.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/0/IMG_5318.JPG
upload: DATA/SIGN_LANGUAGE/valid/0/IMG_5746.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/0/IMG_5746.JPG
upload: DATA/SIGN_LANGUAGE/valid/0/IMG_5507.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/0/IMG_5507.JPG
upload: DATA/SIGN_LANGUAGE/valid/0/IMG_5786.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/0/IMG_5786.JPG
upload: DATA/SIGN_LANGUAGE/valid/0/IMG_5867.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/0/IMG_5867.JPG
upload: DATA/SIGN_LANGUAGE/valid/0/IMG_5938.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/0/IMG_5938.JPG
upload: DATA/SIGN_LANGUAGE/valid/0/IMG_5592.JPG to s3://tf-mme-8

upload: DATA/SIGN_LANGUAGE/valid/3/IMG_4052.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/3/IMG_4052.JPG
upload: DATA/SIGN_LANGUAGE/valid/3/IMG_4708.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/3/IMG_4708.JPG
upload: DATA/SIGN_LANGUAGE/valid/3/IMG_1305.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/3/IMG_1305.JPG
upload: DATA/SIGN_LANGUAGE/valid/3/IMG_4509.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/3/IMG_4509.JPG
upload: DATA/SIGN_LANGUAGE/valid/3/IMG_4489.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/3/IMG_4489.JPG
upload: DATA/SIGN_LANGUAGE/valid/3/IMG_1294.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/3/IMG_1294.JPG
upload: DATA/SIGN_LANGUAGE/valid/3/IMG_4449.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/3/IMG_4449.JPG
upload: DATA/SIGN_LANGUAGE/valid/3/IMG_4043.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/3/IMG_4043.JPG
upload: DATA/SIGN_LANGUAGE/valid/3/IMG_4226.JPG to s3://tf-mme-8

upload: DATA/SIGN_LANGUAGE/valid/5/IMG_4609.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/5/IMG_4609.JPG
upload: DATA/SIGN_LANGUAGE/valid/5/IMG_5168.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/5/IMG_5168.JPG
upload: DATA/SIGN_LANGUAGE/valid/5/IMG_5281.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/5/IMG_5281.JPG
upload: DATA/SIGN_LANGUAGE/valid/5/IMG_5451.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/5/IMG_5451.JPG
upload: DATA/SIGN_LANGUAGE/valid/5/IMG_1153.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/5/IMG_1153.JPG
upload: DATA/SIGN_LANGUAGE/valid/5/IMG_5620.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/5/IMG_5620.JPG
upload: DATA/SIGN_LANGUAGE/valid/5/IMG_5781.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/5/IMG_5781.JPG
upload: DATA/SIGN_LANGUAGE/valid/5/IMG_5913.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/5/IMG_5913.JPG
upload: DATA/SIGN_LANGUAGE/valid/5/IMG_4891.JPG to s3://tf-mme-8

upload: DATA/SIGN_LANGUAGE/valid/8/IMG_1310.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/8/IMG_1310.JPG
upload: DATA/SIGN_LANGUAGE/valid/8/IMG_4067.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/8/IMG_4067.JPG
upload: DATA/SIGN_LANGUAGE/valid/8/IMG_4284.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/8/IMG_4284.JPG
upload: DATA/SIGN_LANGUAGE/valid/8/IMG_4231.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/8/IMG_4231.JPG
upload: DATA/SIGN_LANGUAGE/valid/8/IMG_1136.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/8/IMG_1136.JPG
upload: DATA/SIGN_LANGUAGE/valid/8/IMG_4147.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/8/IMG_4147.JPG
upload: DATA/SIGN_LANGUAGE/valid/8/IMG_4304.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/8/IMG_4304.JPG
upload: DATA/SIGN_LANGUAGE/valid/8/IMG_1247.JPG to s3://tf-mme-892313895307/cv-models/sign_lang/valid/8/IMG_1247.JPG
upload: DATA/SIGN_LANGUAGE/valid/8/IMG_4137.JPG to s3://tf-mme-8

In [15]:
train_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/sign_lang/train', 
                            distribution='ShardedByS3Key')
val_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/sign_lang/valid', 
                          distribution='ShardedByS3Key')

### Estimator 2

In [16]:
model_name = 'model-2'

hyperparameters = {'epochs': 20}
estimator_parameters = {'entry_point':'sign_language_train.py',
                        'instance_type': 'ml.m5.2xlarge',
                        'instance_count': 1,
                        'hyperparameters': hyperparameters,
                        'model_dir': f'/opt/ml/model',
                        'role': role,
                        'output_path': f's3://{BUCKET}/{PREFIX}/sign_lang/out',
                        'base_job_name': f'cv-{model_name}',
                        'framework_version': TF_FRAMEWORK_VERSION,
                        'py_version': 'py37',
                        'script_mode': True}

estimator_2 = TensorFlow(**estimator_parameters)

In [17]:
estimator_2.fit({'train': train_input, 'val': val_input})

2020-11-27 20:44:45 Starting - Starting the training job...
2020-11-27 20:44:47 Starting - Launching requested ML instances......
2020-11-27 20:46:03 Starting - Preparing the instances for training......
2020-11-27 20:46:59 Downloading - Downloading input data......
2020-11-27 20:48:09 Training - Downloading the training image..
2020-11-27 20:48:23 Training - Training image download completed. Training in progress.2020-11-27 20:48:30,405 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2020-11-27 20:48:30,412 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-11-27 20:48:31,074 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-11-27 20:48:31,089 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-11-27 20:48:31,104 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-11-27 20:48:31,114 sagemaker-t

#015 1/18 [>.............................] - ETA: 0s - loss: 7.8639 - accuracy: 0.1000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/18 [==>...........................] - ETA: 6s - loss: 6.2341 - accuracy: 0.2000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 3/18 [====>.........................] - ETA: 7s - loss: 7.9194 - accuracy: 0.1333#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

#015 1/18 [>.............................] - ETA: 0s - loss: 1.2014 - accuracy: 0.7000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/18 [==>...........................] - ETA: 6s - loss: 1.3270 - accuracy: 0.6500#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 3/18 [====>.........................] - ETA: 7s - loss: 1.4150 - accuracy: 0.6000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

#015 1/18 [>.............................] - ETA: 0s - loss: 0.4787 - accuracy: 1.0000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/18 [==>...........................] - ETA: 6s - loss: 0.3298 - accuracy: 1.0000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 3/18 [====>.........................] - ETA: 7s - loss: 0.4906 - accuracy: 0.9000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

#015 1/18 [>.............................] - ETA: 0s - loss: 0.1151 - accuracy: 1.0000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/18 [==>...........................] - ETA: 6s - loss: 0.2192 - accuracy: 0.9000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 3/18 [====>.........................] - ETA: 7s - loss: 0.2006 - accuracy: 0.9333#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

...

#015 1/18 [>.............................] - ETA: 0s - loss: 0.0408 - accuracy: 1.0000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/18 [==>...........................] - ETA: 5s - loss: 0.0367 - accuracy: 1.0000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 3/18 [====>.........................] - ETA: 7s - loss: 0.0481 - accuracy: 1.0000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

#015 1/18 [>.............................] - ETA: 0s - loss: 0.0192 - accuracy: 1.0000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/18 [==>...........................] - ETA: 6s - loss: 0.0331 - accuracy: 1.0000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 3/18 [====>.........................] - ETA: 7s - loss: 0.0276 - accuracy: 1.0000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

#015 1/18 [>.............................] - ETA: 0s - loss: 0.0022 - accuracy: 1.0000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/18 [==>...........................] - ETA: 5s - loss: 0.0028 - accuracy: 1.0000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 3/18 [====>.........................] - ETA: 7s - loss: 0.0021 - accuracy: 1.0000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

#015 1/18 [>.............................] - ETA: 0s - loss: 0.2619 - accuracy: 0.9000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/18 [==>...........................] - ETA: 6s - loss: 0.1330 - accuracy: 0.9500#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 3/18 [====>.........................] - ETA: 7s - loss: 0.0926 - accuracy: 0.9667#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0


2020-11-27 20:55:05 Uploading - Uploading generated training model
2020-11-27 20:55:23 Completed - Training job completed
Training seconds: 504
Billable seconds: 504


## Multi-Model Endpoint (MME)

In [18]:
tf_model_1 = estimator_1.model_data
output_1 = f's3://{BUCKET}/{PREFIX}/mme/model-1.tar.gz'

In [19]:
tf_model_2 = estimator_2.model_data
output_2 = f's3://{BUCKET}/{PREFIX}/mme/model-2.tar.gz'

In [20]:
!aws s3 cp {tf_model_1} {output_1} 
!aws s3 cp {tf_model_2} {output_2} 

copy: s3://tf-mme-892313895307/cv-models/cifar/out/cv-model-1-2020-11-27-20-14-12-336/output/model.tar.gz to s3://tf-mme-892313895307/cv-models/mme/model-1.tar.gz
copy: s3://tf-mme-892313895307/cv-models/sign_lang/out/cv-model-2-2020-11-27-20-44-45-298/output/model.tar.gz to s3://tf-mme-892313895307/cv-models/mme/model-2.tar.gz


### Deploy TensorFlow Model-1 as a Multi-Model Endpoint 
<p>https://github.com/aws/deep-learning-containers/blob/master/available_images.md</p>

In [21]:
current_time = datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d-%H-%M-%S')
current_time

'2020-11-27-22-42-28'

In [22]:
IMAGE_URI = '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.3.1-cpu-py37-ubuntu18.04'
model_data_prefix = f's3://{BUCKET}/{PREFIX}/mme/'
model_1 = TensorFlowModel(model_data=output_1, 
                          role=role, 
                          image_uri=IMAGE_URI)

In [23]:
mme = MultiDataModel(name=f'mme-tensorflow-{current_time}',
                     model_data_prefix=model_data_prefix,
                     model=model_1,
                     sagemaker_session=sagemaker_session)

In [24]:
predictor = mme.deploy(initial_instance_count=1,
                       instance_type='ml.m5.2xlarge',
                       endpoint_name=f'mme-tensorflow-{current_time}')

-------------!

### Test MME

In [25]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [26]:
list(mme.list_models())

['model-1.tar.gz', 'model-2.tar.gz']

In [27]:
img_path = './DATA/CIFAR_10/raw_images/truck.png'
img = image.load_img(img_path)
X = image.img_to_array(img)
X = X.astype('float32')/255
X = X.reshape(1, 32, 32, 3)

ValueError: cannot reshape array of size 373248 into shape (1,32,32,3)

In [ ]:
payload = {'instances': X}

In [ ]:
predictor.predict(data=payload, initial_args={'TargetModel': 'model1.tar.gz'})

#### Test Model 2

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
img_path = './DATA/SIGN_LANGUAGE/test/0/IMG_1290.JPG'
img = image.load_img(img_path, target_size=(224, 224))
X = image.img_to_array(img)
X = np.expand_dims(X, axis=0)

In [ ]:
X.shape

In [ ]:
payload = {'instances': X}

In [ ]:
predictor.predict(data=payload, initial_args={'TargetModel': 'model2.tar.gz'})